<a href="https://colab.research.google.com/github/bo-cheng-tsai/homework/blob/main/week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [6]:
!pip install datasets transformers

In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from evaluate import load

# 下載金融文本資料集
dataset = load_dataset("takala/financial_phrasebank", "sentences_50agree")

# 將資料分為訓練集和測試集
train_test_split = dataset["train"].train_test_split(test_size=0.2)
dataset = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

# 載入 BERT 分詞器和模型
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3 個情緒類別

# 預處理數據
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True, max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# 加載準確率計算指標
metric = load("accuracy")

# 定義計算指標的函數
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# 定義訓練參數，修正 eval_steps 和 save_steps
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # 使用 steps 評估策略
    save_strategy="steps",       # 與 evaluation_strategy 保持一致
    eval_steps=100,              # 設置評估間隔步數
    save_steps=100,              # 設置保存間隔步數（必須為 eval_steps 的倍數）
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True, # 啟用加載最佳模型
    report_to="none"
)

# 定義 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 訓練模型
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-d28ed351316a>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,No log,0.520079,0.784536
200,No log,0.414703,0.835052
300,No log,0.427990,0.838144
400,No log,0.485427,0.836082
500,0.411400,0.478099,0.840206
600,0.411400,0.502821,0.855670
700,0.411400,0.564464,0.847423
800,0.411400,0.623721,0.855670
900,0.411400,0.640247,0.855670
1000,0.109200,0.676075,0.857732


TrainOutput(global_step=1215, training_loss=0.22183464270069767, metrics={'train_runtime': 623.3424, 'train_samples_per_second': 31.09, 'train_steps_per_second': 1.949, 'total_flos': 1269974466724320.0, 'train_loss': 0.22183464270069767, 'epoch': 5.0})

In [16]:
# 評估模型
results = trainer.evaluate()
print(f"Test Accuracy: {results['eval_accuracy']:.4f}")

Test Accuracy: 0.8351


In [17]:
# 測試句子
test_texts = [
    "The company's profit has increased significantly this quarter.",  # Positive
    "The increase in costs negatively affected the revenue.",  # Negative
    "The company's performance remained stable."  # Neutral
]
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# 推論
model.eval()
outputs = model(**test_encodings)
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# 將數字標籤轉換為文字標籤
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
predicted_labels = [label_map[pred] for pred in preds]
print(predicted_labels)

['Positive', 'Negative', 'Positive']
